In [34]:
!ls

app.py	apps  assets  index.py	plotly_dash.ipynb  __pycache__


In [33]:
import pandas as pd

df = pd.read_csv('/home/kyle/Desktop/NHL-Salary-Predictions/data/final_cleaned_player_data_dash.csv')

In [34]:
single_player = df[df['Player Name'].str.contains('P.K.')]

In [35]:
single_player

,id,Player Name,link,Position,Age,Nationality,Height,Weight,Shoots,Time On Ice,...,games22_quantile,shifts22_quantile,blocked22_quantile,timeOnIcePerGame22_quantile,evenTimeOnIcePerGame22_quantile,shortHandedTimeOnIcePerGame22_quantile,powerPlayTimeOnIcePerGame22_quantile,overall_rank_sum,Overall Rank,Height (Inches)
0,P.K. Subban,P.K. Subban,/api/v1/people/8474056,Defenseman,32,CAN,"6' 0""",210,Right,1408.48,...,3,3,4,3,4,2,2,695,295,72.0


In [36]:
for col in single_player.columns:
    if 'Time' in col and 'quantile' not in col:
        single_player[col] = single_player[col].astype(str).str.replace('.', ':', regex=True) + ' Min'

In [37]:
single_player

,id,Player Name,link,Position,Age,Nationality,Height,Weight,Shoots,Time On Ice,...,games22_quantile,shifts22_quantile,blocked22_quantile,timeOnIcePerGame22_quantile,evenTimeOnIcePerGame22_quantile,shortHandedTimeOnIcePerGame22_quantile,powerPlayTimeOnIcePerGame22_quantile,overall_rank_sum,Overall Rank,Height (Inches)
0,P.K. Subban,P.K. Subban,/api/v1/people/8474056,Defenseman,32,CAN,"6' 0""",210,Right,1408:48 Min,...,3,3,4,3,4,2,2,695,295,72.0


In [38]:
import numpy as np

In [55]:
data_label_ = np.full_like(single_player['Player Name'], str('Assists'))

customdata = np.stack((single_player['Player Name'], single_player['Overall Rank'], single_player['Salary Rank'], data_label_),axis=-1)
print(customdata[0][0])
hover_template = "<b>Player Name: </b> %{customdata[0][0]} <br><br>"
# hover_template += "<b>%{customdata[2]}: </b> %{x} <br>"
# hover_template += "<b>Salary: </b> $%{y} <br>"
# hover_template += "<b>Salary Rank: </b> %{customdata[1]} <br>"
# hover_template += "<b>Player Rank: </b> %{customdata[0]}"

P.K. Subban


In [56]:
import chart_studio.plotly as py
import plotly.figure_factory as ff
import numpy as np

In [57]:
# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot([df['Total Assists']], ['Assists'], show_hist=False)

# Add title
fig.update_traces(hovertemplate=hover_template, customdata=customdata)
fig.update_layout(showlegend=False, margin=dict(l=0, r=0, t=0, b=0))
fig.update_yaxes(visible=False)
fig.update_xaxes(visible=False)
fig.add_vline(x=df[df['Player Name'].str.contains('P.K.')]['Total Assists'][0], line_color='yellow')
fig.show()

In [62]:
fig.data[0]['hovertemplate'] = "<b>Player Name: </b> P.K. Subban} <br><br>"

In [59]:
testing = np.array([['P.K. Subban', 295, 23, 'Assists']], dtype='object')

In [50]:
testing[0][0]

'P.K. Subban'